In [7]:
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from datetime import datetime
from pathlib import Path
import time
from ultralytics import YOLO  # YOLOv8

class RealTimeStreamSimulatorYOLOv8:
    def __init__(self, model_path='yolov8x.pt', confidence_threshold=0.5):
        print(f"Loading {model_path} model for real-time simulation...")
        self.model = YOLO(model_path)
        self.model.conf = confidence_threshold
        self.person_class_id = 0  # 'person' class index in COCO

        self.consecutive_crowd_frames = deque(maxlen=5)
        self.alerts = []
        self.alert_count = 0
        print("✓ YOLOv8 model loaded. Monitoring for crowd alerts...")

    def simulate_stream(self, video_path, output_dir='task2_outputs', save_overlay=True):
        Path(output_dir).mkdir(exist_ok=True)
        cap = cv2.VideoCapture(video_path)

        if not cap.isOpened():
            raise ValueError(f"Could not open video: {video_path}")

        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        if save_overlay:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out_path = str(Path(output_dir) / 'stream_with_alerts.mp4')
            writer = cv2.VideoWriter(out_path, fourcc, fps / 3, (width, height))

        print(f"Video: {video_path} | FPS: {fps} | Total Frames: {total_frames}")
        print("Processing every 3rd frame...")

        frame_idx = 0
        processed = 0
        start_time = time.time()

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            if frame_idx % 3 == 0:
                timestamp = frame_idx / fps
                people_count, detections = self._detect_people(frame)
                alert_triggered = self._check_crowd_alert(people_count, timestamp, frame_idx)
                overlay = self._create_overlay(frame, detections, people_count, alert_triggered)
                if save_overlay:
                    writer.write(overlay)
                processed += 1

            frame_idx += 1

        cap.release()
        if save_overlay:
            writer.release()

        self._save_alert_logs(output_dir)
        self._create_timeline_visualization(output_dir)

        print(f"\n✅ Simulation finished. Total alerts: {len(self.alerts)}")
        return self.alerts

    def _detect_people(self, frame):
        results = self.model(frame, verbose=False)[0]  # first batch

        people_detections = []
        people_count = 0

        for box in results.boxes:
            class_id = int(box.cls[0].item())
            conf = float(box.conf[0].item())

            if class_id == self.person_class_id:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                people_count += 1
                people_detections.append({
                    'confidence': conf,
                    'bbox': {
                        'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2
                    }
                })

        return people_count, people_detections

    def _check_crowd_alert(self, count, timestamp, frame_number):
        self.consecutive_crowd_frames.append(count >= 3)
        if len(self.consecutive_crowd_frames) == 5 and all(self.consecutive_crowd_frames):
            if not hasattr(self, '_last_alert_frame') or frame_number - self._last_alert_frame > 15:
                self.alert_count += 1
                alert = {
                    'alert_id': self.alert_count,
                    'timestamp': timestamp,
                    'frame_number': frame_number,
                    'people_count': count,
                    'message': f"Crowd Detected: {count} people",
                    'datetime': datetime.now().isoformat()
                }
                self.alerts.append(alert)
                self._last_alert_frame = frame_number
                print(f"🚨 ALERT #{self.alert_count}: {count} people at {timestamp:.2f}s")
                return True
        return False

    def _create_overlay(self, frame, detections, count, alert):
        overlay = frame.copy()
        for det in detections:
            x1, y1, x2, y2 = map(int, [det['bbox']['x1'], det['bbox']['y1'], det['bbox']['x2'], det['bbox']['y2']])
            color = (0, 0, 255) if alert else (0, 255, 0)
            cv2.rectangle(overlay, (x1, y1), (x2, y2), color, 2)
            label = f"Person: {det['confidence']:.2f}"
            cv2.putText(overlay, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        cv2.putText(overlay, f"People Count: {count}", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        if alert:
            cv2.putText(overlay, "🚨 CROWD ALERT 🚨", (20, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

        consecutive = sum(self.consecutive_crowd_frames)
        cv2.putText(overlay, f"Consecutive Crowd Frames: {consecutive}/5",
                    (20, overlay.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        return overlay

    def _save_alert_logs(self, output_dir):
        output_dir = Path(output_dir)
        json_path = output_dir / 'alert_logs.json'
        txt_path = output_dir / 'alert_logs.txt'

        with open(json_path, 'w') as f:
            json.dump({'total_alerts': len(self.alerts), 'alerts': self.alerts}, f, indent=2)

        with open(txt_path, 'w') as f:
            f.write("CROWD ALERT LOG\n" + "="*40 + "\n\n")
            for alert in self.alerts:
                f.write(f"Alert #{alert['alert_id']} - Frame: {alert['frame_number']}, People: {alert['people_count']}, Time: {alert['timestamp']:.2f}s\n")

        print(f"✓ Alerts saved to {json_path} and {txt_path}")

    def _create_timeline_visualization(self, output_dir):
        if not self.alerts:
            print("No alerts to visualize.")
            return

        timestamps = [a['timestamp'] for a in self.alerts]
        people_counts = [a['people_count'] for a in self.alerts]

        plt.figure(figsize=(12, 6))
        plt.scatter(timestamps, people_counts, c='red', s=80, label='Alert')
        plt.plot(timestamps, people_counts, linestyle='--', alpha=0.5)
        plt.xlabel("Time (s)")
        plt.ylabel("People Count")
        plt.title("Crowd Alert Timeline")
        plt.grid(True)
        plt.legend()
        plot_path = Path(output_dir) / 'alert_timeline.png'
        plt.savefig(plot_path)
        plt.close()
        print(f"✓ Timeline saved to {plot_path}")

# Main function
if __name__ == "__main__":
    simulator = RealTimeStreamSimulatorYOLOv8(
        model_path='yolov8x.pt',  # Make sure it's downloaded or available
        confidence_threshold=0.5
    )
    simulator.simulate_stream(
        video_path="/content/Orginal video sample for Automatic Counting People System.mp4",  # Replace with your actual path
        output_dir="task2_outputs_8x",
        save_overlay=True
    )


Loading yolov8x.pt model for real-time simulation...
✓ YOLOv8 model loaded. Monitoring for crowd alerts...
Video: /content/Orginal video sample for Automatic Counting People System.mp4 | FPS: 30.0 | Total Frames: 698
Processing every 3rd frame...
🚨 ALERT #1: 3 people at 0.60s
🚨 ALERT #2: 4 people at 1.20s
🚨 ALERT #3: 4 people at 1.80s
🚨 ALERT #4: 4 people at 2.40s
🚨 ALERT #5: 4 people at 3.00s
🚨 ALERT #6: 4 people at 3.60s
🚨 ALERT #7: 5 people at 4.20s
🚨 ALERT #8: 4 people at 4.80s
🚨 ALERT #9: 3 people at 6.00s
🚨 ALERT #10: 3 people at 8.70s
🚨 ALERT #11: 4 people at 9.30s
🚨 ALERT #12: 5 people at 9.90s
🚨 ALERT #13: 6 people at 10.50s
🚨 ALERT #14: 7 people at 11.10s
🚨 ALERT #15: 8 people at 11.70s
🚨 ALERT #16: 12 people at 12.30s
🚨 ALERT #17: 8 people at 12.90s
🚨 ALERT #18: 7 people at 13.50s
🚨 ALERT #19: 7 people at 14.10s
🚨 ALERT #20: 7 people at 14.70s
🚨 ALERT #21: 6 people at 15.30s
🚨 ALERT #22: 5 people at 15.90s
🚨 ALERT #23: 6 people at 16.50s
🚨 ALERT #24: 5 people at 17.10s
🚨 ALE